### Домашнее задание по теме 10

### Задание 1


In [2]:
import pytest

def check_palindrom(string_: str):
    """Функция убирает пробелы и 
    переводит символы в нижний регистр в строке
    затем проверяет ее на валидность палиндрома
    и в зависимости от этого возвращает сообщение"""
    a=string_.lower().strip()
    if a==a[::-1]:
        return 'строка есть палиндром'
    else:
        return 'строка не полиндром'


# тестирование функции если строка палиндром
@pytest.mark.parametrize('string_,palindrom', [('топот','строка есть палиндром'),('sos','строка есть палиндром'),('Казак','строка есть палиндром')])
def test_check_palindrom(string_, palindrom):
    assert check_palindrom(string_)==palindrom
    
# тестирование функции если строка не палиндром
@pytest.mark.parametrize('string_2, not_palindrom', [('ропот','строка не полиндром'), (' ЦокоТ','строка не полиндром'),('gor','строка не полиндром')]) 
def test_not_check_palindrom(string_2, not_palindrom):
     assert check_palindrom(string_2)==not_palindrom     

### Задание 2

In [4]:
class Employee:
    def __init__(self, name, seniority):
        self.name = name
        self.seniority = seniority
        
        self.grade = 1
    
    def grade_up(self):
        """Повышает уровень сотрудника"""
        self.grade += 1
    
    def publish_grade(self):
        """Публикация результатов аккредитации сотрудников"""
        print(self.name, self.grade)
    
    def check_if_it_is_time_for_upgrade(self):
        pass
    
class Designer(Employee):
    def __init__(self, name, seniority,international_award=2): # дизайнер изначально имеет 2 премии
        super().__init__(name, seniority)
        self.international_award=international_award
        """ добавление по 2 балла за 1 премию """ 
        self.seniority=self.international_award*2+self.seniority
    
    def check_if_it_is_time_for_upgrade(self):
        
        # для каждой аккредитации увеличиваем счетчик на 1
        # пока считаем, что все дизайнеры проходят аккредитацию
        self.seniority+=1
        
        # условие повышения сотрудника после 7 баллов
        if self.seniority % 7 == 0:
            self.grade_up()
        
        # публикация результатов
        return self.publish_grade()
class Developer(Employee):
    def __init__(self, name, seniority):
        super().__init__(name, seniority)
    
    def check_if_it_is_time_for_upgrade(self):
        # для каждой аккредитации увеличиваем счетчик на 1
        # пока считаем, что все разработчики проходят аккредитацию
        self.seniority += 1
        
        # условие повышения сотрудника из презентации
        if self.seniority % 5 == 0:
            self.grade_up()
        
        # публикация результатов
        return self.publish_grade()
    

import pytest

@pytest.fixture
def designer_fixture():
    NAME_DESIGNER='Сергей'
    def _init_designer(seniority):
            return Designer(NAME_DESIGNER,seniority=seniority)
    return _init_designer

@pytest.fixture
def developer_fixture():
    NAME_DEVELOPER='Aleksey'
    def _init_developer(seniority):
        return Developer(NAME_DEVELOPER,seniority=seniority)
    return _init_developer
    
@pytest.mark.parametrize('enter,exeption',[(0,1),(4,2),(9,3),(14,4)])
def test_developer_metod(developer_fixture,enter,exeption):
    dev=developer_fixture(2)
    for i in range(enter):
        dev.check_if_it_is_time_for_upgrade()
    assert dev.grade==exeption        

@pytest.mark.parametrize('enter,exeption',[(0,1),(1,2),(2,2),(9,3)])
def test_designer_metod(designer_fixture,enter,exeption):
    des=designer_fixture(2)
    for i in range(enter):
        des.check_if_it_is_time_for_upgrade()
    assert des.grade==exeption 
    

### Задание 3

In [5]:
import requests
class Rate:
   
    def __init__(self, format_='value', diff=False):
        self.format = format_
        self.diff=diff      #добавлена переменная diff
    
    def exchange_rates(self):
        
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        self.r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return self.r.json()['Valute']
    def make_format(self, currency):
        """
        Возвращает информацию о валюте currency в трех вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        Rate('value').make_format('EUR')
        79.4966
        
        Rate('value',True) выводит разницу с предыдущим курсом
        """
        response = self.exchange_rates()
        
        if currency in response:
            if self.format == 'full':
                return response[currency]
            
            if self.format == 'value' and self.diff==False:
                return response[currency]['Value']
            if self.format == 'value' and self.diff==True:
                return response [currency]['Value']-response [currency]['Previous']
        
        return 'Error'
    
    def eur(self):
        """Возвращает курс евро на сегодня в формате self.format"""
        return self.make_format('EUR')
    
    def usd(self):
        """Возвращает курс доллара на сегодня в формате self.format"""
        return self.make_format('USD')
    
    def brl(self):
        """Возвращает курс бразильского реала на сегодня в формате self.format"""
        return self.make_format('BRL')
    
    
    def max_exchange(self):
        """
        функция возвращающая название валюты с максимальным значением курса
        """
        
        name_max_exchange='' # переменная с названием валюты с максимальным курсом
        response = self.exchange_rates() # получаем словарь с курсами всех валют
        list_value_currency=[] # переменная списка курсов валют
        for k in response.keys():  # в цикле перебираем и записываем значения курса каждой валюты в список 
            list_value_currency.append(response.get(k)['Value']/response.get(k)['Nominal']) # для получения курса делим на номинал валюты
        max_exchange=max(list_value_currency) # получаем максимальное значение валюты из списка
        
        for k in response.keys():  # в цикле перебираем значения всех валют пока не будет совпадения с максимальным значением
            if response.get(k)['Value']==max_exchange:
                name_max_exchange=response.get(k)['Name'] # сохраняем название валюты с максимальным значением
                break
        return name_max_exchange
        
        
        
import pytest
@pytest.fixture
def rate_fixture():
    def _init_rate(format_,diff):
        return Rate(format_=format_,diff=diff)
    return _init_rate

full_value={
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode':'978',
            'Previous':79.6765,
            'Value':79.4966
                }
@pytest.mark.parametrize('format_,diff,expected',[('full',False,full_value),('full',True,full_value),('value',False,79.4966),('value',True,-0.1799000000000035)])
def test_exchange(rate_fixture,mocker,format_,diff,expected):
    mocker.patch('test_10_3.Rate.exchange_rates',return_value={
            'EUR': {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode':'978',
            'Previous':79.6765,
            'Value': 79.4966
                }})    
    c=rate_fixture(format_,diff) 
    assert c.eur()==expected
   